In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Daftar class makanan 
food_classes = [
    "AW cola", "Beijing Beef", "Chow Mein", "Fried Rice", "Hashbrown", "Honey Walnut Shrimp", "Kung Pao Chicken",
    "String Bean Chicken Breast", "Super Greens", "The Original Orange Chicken", "White Steamed Rice", 
    "Black Pepper Rice Bowl", "Burger", "Carrot Eggs", "Cheese Burger", "Chicken Waffle", "Chicken Nuggets", 
    "Chinese Cabbage", "Chinese Sausage", "Crispy Corn", "Curry", "French Fries", "Fried Chicken", "Fried Dumplings", 
    "Fried Eggs", "Mango Chicken Pocket", "Mozza Burger", "Mung Bean Sprouts", "Nugget", "Perkedel", "Rice", 
    "Sprite", "Tostitos Cheese Dip Sauce", "Triangle Hash Brown", "Water Spinach"
]
num_classes = len(food_classes)

# Nilai nutrisi dummy untuk setiap class makanan
nutritional_values = {
    "AW cola": {"kalori": 150, "protein": 0, "lemak": 0, "karbohidrat": 40, "serat": 0, "vitamin_C": 0, "kalsium": 0},
    "Beijing Beef": {"kalori": 470, "protein": 19, "lemak": 25, "karbohidrat": 41, "serat": 2, "vitamin_C": 0, "kalsium": 0},
    "Chow Mein": {"kalori": 510, "protein": 14, "lemak": 20, "karbohidrat": 70, "serat": 4, "vitamin_C": 1, "kalsium": 30},
    "Fried Rice": {"kalori": 520, "protein": 11, "lemak": 16, "karbohidrat": 85, "serat": 3, "vitamin_C": 1, "kalsium": 20},
    "Hashbrown": {"kalori": 150, "protein": 1, "lemak": 10, "karbohidrat": 15, "serat": 2, "vitamin_C": 0, "kalsium": 0},
    "Honey Walnut Shrimp": {"kalori": 360, "protein": 13, "lemak": 23, "karbohidrat": 28, "serat": 1, "vitamin_C": 2, "kalsium": 40},
    "Kung Pao Chicken": {"kalori": 290, "protein": 14, "lemak": 19, "karbohidrat": 13, "serat": 3, "vitamin_C": 8, "kalsium": 20},
    "String Bean Chicken Breast": {"kalori": 190, "protein": 14, "lemak": 9, "karbohidrat": 12, "serat": 4, "vitamin_C": 15, "kalsium": 40},
    "Super Greens": {"kalori": 90, "protein": 6, "lemak": 5, "karbohidrat": 7, "serat": 5, "vitamin_C": 60, "kalsium": 70},
    "The Original Orange Chicken": {"kalori": 490, "protein": 13, "lemak": 23, "karbohidrat": 51, "serat": 2, "vitamin_C": 0, "kalsium": 20},
    "White Steamed Rice": {"kalori": 380, "protein": 7, "lemak": 0, "karbohidrat": 86, "serat": 1, "vitamin_C": 0, "kalsium": 0},
    "Black Pepper Rice Bowl": {"kalori": 420, "protein": 15, "lemak": 10, "karbohidrat": 62, "serat": 2, "vitamin_C": 0, "kalsium": 0},
    "Burger": {"kalori": 530, "protein": 21, "lemak": 27, "karbohidrat": 50, "serat": 2, "vitamin_C": 1, "kalsium": 100},
    "Carrot Eggs": {"kalori": 180, "protein": 8, "lemak": 12, "karbohidrat": 10, "serat": 3, "vitamin_C": 4, "kalsium": 50},
    "Cheese Burger": {"kalori": 600, "protein": 23, "lemak": 33, "karbohidrat": 55, "serat": 3, "vitamin_C": 1, "kalsium": 150},
    "Chicken Waffle": {"kalori": 400, "protein": 17, "lemak": 23, "karbohidrat": 30, "serat": 2, "vitamin_C": 0, "kalsium": 60},
    "Chicken Nuggets": {"kalori": 290, "protein": 15, "lemak": 19, "karbohidrat": 16, "serat": 1, "vitamin_C": 0, "kalsium": 10},
    "Chinese Cabbage": {"kalori": 20, "protein": 1, "lemak": 0, "karbohidrat": 4, "serat": 2, "vitamin_C": 45, "kalsium": 40},
    "Chinese Sausage": {"kalori": 340, "protein": 10, "lemak": 25, "karbohidrat": 17, "serat": 1, "vitamin_C": 0, "kalsium": 15},
    "Crispy Corn": {"kalori": 200, "protein": 3, "lemak": 12, "karbohidrat": 25, "serat": 2, "vitamin_C": 6, "kalsium": 10},
    "Curry": {"kalori": 300, "protein": 10, "lemak": 20, "karbohidrat": 20, "serat": 4, "vitamin_C": 10, "kalsium": 30},
    "French Fries": {"kalori": 380, "protein": 4, "lemak": 18, "karbohidrat": 50, "serat": 4, "vitamin_C": 6, "kalsium": 15},
    "Fried Chicken": {"kalori": 430, "protein": 24, "lemak": 25, "karbohidrat": 20, "serat": 2, "vitamin_C": 0, "kalsium": 30},
    "Fried Dumplings": {"kalori": 230, "protein": 10, "lemak": 12, "karbohidrat": 20, "serat": 1, "vitamin_C": 1, "kalsium": 20},
    "Fried Eggs": {"kalori": 100, "protein": 7, "lemak": 8, "karbohidrat": 1, "serat": 0, "vitamin_C": 0, "kalsium": 20},
    "Mango Chicken Pocket": {"kalori": 400, "protein": 15, "lemak": 18, "karbohidrat": 50, "serat": 4, "vitamin_C": 30, "kalsium": 40},
    "Mozza Burger": {"kalori": 600, "protein": 28, "lemak": 35, "karbohidrat": 45, "serat": 2, "vitamin_C": 1, "kalsium": 200},
    "Mung Bean Sprouts": {"kalori": 31, "protein": 3, "lemak": 0.2, "karbohidrat": 6, "serat": 2, "vitamin_C": 13, "kalsium": 14},
    "Nugget": {"kalori": 270, "protein": 14, "lemak": 17, "karbohidrat": 15, "serat": 1, "vitamin_C": 0, "kalsium": 10},
    "Perkedel": {"kalori": 200, "protein": 4, "lemak": 12, "karbohidrat": 20, "serat": 2, "vitamin_C": 0, "kalsium": 20},
    "Rice": {"kalori": 130, "protein": 2.5, "lemak": 0.3, "karbohidrat": 28, "serat": 0.3, "vitamin_C": 0, "kalsium": 3},
    "Sprite": {"kalori": 140, "protein": 0, "lemak": 0, "karbohidrat": 38, "serat": 0, "vitamin_C": 0, "kalsium": 0},
    "Tostitos Cheese Dip Sauce": {"kalori": 60, "protein": 1, "lemak": 4, "karbohidrat": 2, "serat": 0, "vitamin_C": 0, "kalsium": 40},
    "Triangle Hash Brown": {"kalori": 150, "protein": 1, "lemak": 10, "karbohidrat": 15, "serat": 2, "vitamin_C": 0, "kalsium": 0},
    "Water Spinach": {"kalori": 40, "protein": 3, "lemak": 0.5, "karbohidrat": 8, "serat": 2.5, "vitamin_C": 50, "kalsium": 30}
}

def calculate_bmr(age, weight, height, gender):
    """
    Menghitung Basal Metabolic Rate (BMR) berdasarkan parameter pengguna.
    
    Args:
        age (int): Usia pengguna
        weight (float): Berat badan pengguna (dalam kg)
        height (float): Tinggi badan pengguna (dalam cm)
        gender (str): Jenis kelamin pengguna ('male' atau 'female')
        
    Returns:
        float: BMR dari pengguna
        
    Raises:
        ValueError: Jika gender bukan 'male' atau 'female'
    """
    if gender.lower() == "male":
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    elif gender.lower() == "female":
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    else:
        raise ValueError("Gender harus 'male' atau 'female'.")
    return bmr


def calculate_tdee(bmr, activity_level):
    """
    Menghitung Total Daily Energy Expenditure (TDEE) berdasarkan BMR dan level aktivitas.

    Args:
        bmr (float): Nilai Basal Metabolic Rate (BMR)
        activity_level (float): Faktor aktivitas fisik (1.2 - 1.9)

    Returns:
        float: Nilai TDEE
    """
    return bmr * activity_level


def create_dummy_dataset_with_tdee(num_samples=100):
    """
    Membuat dataset dummy yang berisi informasi pengguna, BMR, dan TDEE berdasarkan sampel acak.

    Args:
        num_samples (int): Jumlah sampel yang ingin dihasilkan

    Returns:
        list[dict]: List berisi data pengguna termasuk umur, berat, tinggi, gender, aktivitas, BMR, dan TDEE.
    """
    user_data = {
        "user_age": np.arange(20, 60),
        "user_weight": np.arange(50, 100),
        "user_height": np.arange(150, 200),
        "user_gender": ["male", "female"],
        "activity_level": [1.2, 1.375, 1.55, 1.725, 1.9]  # Level aktivitas
    }
    
    data = []
    for _ in range(num_samples):
        user_age = np.random.choice(user_data["user_age"])
        user_weight = np.random.choice(user_data["user_weight"])
        user_height = np.random.choice(user_data["user_height"])
        user_gender = np.random.choice(user_data["user_gender"])
        activity_level = np.random.choice(user_data["activity_level"])

        bmr = calculate_bmr(user_age, user_weight, user_height, user_gender)
        tdee = calculate_tdee(bmr, activity_level)

        data.append({
            "user_age": user_age,
            "user_weight": user_weight,
            "user_height": user_height,
            "user_gender": user_gender,
            "activity_level": activity_level,
            "tdee": tdee,
        })
    return data


class ComplexDailyIntakeMemoryWithFeedback:
    """
    Memori harian kompleks yang mengelola konsumsi kalori, makronutrien, serta memberikan rekomendasi makanan
    dan menerima feedback dari pengguna.
    """
    
    def __init__(self, target_calories, target_protein, target_fat, target_carbs, target_fiber, target_vitamin_C, target_calcium):
        """
        Inisialisasi memori konsumsi harian dengan target nutrisi harian pengguna.
        """
        self.target_calories = target_calories
        self.target_protein = target_protein
        self.target_fat = target_fat
        self.target_carbs = target_carbs
        self.target_fiber = target_fiber
        self.target_vitamin_C = target_vitamin_C
        self.target_calcium = target_calcium
        self.consumed_calories = 0
        self.consumed_protein = 0
        self.consumed_fat = 0
        self.consumed_carbs = 0
        self.consumed_fiber = 0
        self.consumed_vitamin_C = 0
        self.consumed_calcium = 0
        self.meal_history = []  # List untuk menyimpan makanan dan waktu makan
        self.recommended_history = []  # Track rekomendasi sebelumnya untuk menghindari pengulangan

    def add_food(self, food_name, food_weight, meal_time):
        """
        Menambahkan makanan yang dikonsumsi ke dalam memori dengan menghitung nilai nutrisi yang dikonsumsi.

        Args:
            food_name (str): Nama makanan
            food_weight (float): Berat makanan dalam gram
            meal_time (str): Waktu makan (misalnya 'sarapan', 'makan siang')
        """
        if food_name in nutritional_values:
            nutrisi = nutritional_values[food_name]
            self.consumed_calories += (nutrisi["kalori"] / 100) * food_weight
            self.consumed_protein += (nutrisi["protein"] / 100) * food_weight
            self.consumed_fat += (nutrisi["lemak"] / 100) * food_weight
            self.consumed_carbs += (nutrisi["karbohidrat"] / 100) * food_weight
            self.consumed_fiber += (nutrisi.get("serat", 0) / 100) * food_weight
            self.consumed_vitamin_C += (nutrisi.get("vitamin_C", 0) / 100) * food_weight
            self.consumed_calcium += (nutrisi.get("kalsium", 0) / 100) * food_weight
            self.meal_history.append((food_name, meal_time))

    def remaining_intake(self):
        """
        Menghitung sisa target nutrisi yang belum terpenuhi berdasarkan konsumsi saat ini.

        Returns:
            dict: Mengembalikan sisa kalori, protein, lemak, karbohidrat, serat, vitamin C, dan kalsium.
        """
        return {
            "remaining_calories": self.target_calories - self.consumed_calories,
            "remaining_protein": self.target_protein - self.consumed_protein,
            "remaining_fat": self.target_fat - self.consumed_fat,
            "remaining_carbs": self.target_carbs - self.consumed_carbs,
            "remaining_fiber": self.target_fiber - self.consumed_fiber,
            "remaining_vitamin_C": self.target_vitamin_C - self.consumed_vitamin_C,
            "remaining_calcium": self.target_calcium - self.consumed_calcium
        }

    def score_food(self, food_nutrients, remaining):
        """
        Menghitung skor kecocokan makanan dengan sisa kebutuhan nutrisi.
        
        Args:
            food_nutrients (dict): Nutrisi makanan yang sedang dievaluasi
            remaining (dict): Sisa kebutuhan nutrisi pengguna

        Returns:
            float: Skor kecocokan makanan
        """
        score = 0
        score += (1 - abs(food_nutrients["kalori"] - remaining["remaining_calories"]) / remaining["remaining_calories"]) * 0.4
        score += (1 - abs(food_nutrients["protein"] - remaining["remaining_protein"]) / remaining["remaining_protein"]) * 0.3
        score += (1 - abs(food_nutrients["lemak"] - remaining["remaining_fat"]) / remaining["remaining_fat"]) * 0.2
        score += (1 - abs(food_nutrients["karbohidrat"] - remaining["remaining_carbs"]) / remaining["remaining_carbs"]) * 0.1
        return score

    def dynamic_adjustment(self, meal_time, use_collaborative_filtering=False):
        """
        Mengatur rekomendasi makanan berdasarkan sisa kebutuhan nutrisi dan riwayat makanan yang dikonsumsi.

        Args:
            meal_time (str): Waktu makan untuk rekomendasi
            use_collaborative_filtering (bool): Menggunakan collaborative filtering atau tidak
        """
        remaining = self.remaining_intake()
        print(f"\nKebutuhan sisa untuk {meal_time}:")
        print(f"Kalori tersisa: {remaining['remaining_calories']:.2f} kcal")
        print(f"Protein tersisa: {remaining['remaining_protein']:.2f} gram")
        print(f"Lemak tersisa: {remaining['remaining_fat']:.2f} gram")
        print(f"Karbohidrat tersisa: {remaining['remaining_carbs']:.2f} gram")
        print(f"Serat tersisa: {remaining['remaining_fiber']:.2f} gram")
        print(f"Vitamin C tersisa: {remaining['remaining_vitamin_C']:.2f} mg")
        print(f"Kalsium tersisa: {remaining['remaining_calcium']:.2f} mg")

        # Menyimpan rekomendasi yang memenuhi kriteria
        recommended_foods = []
        scored_foods = []

        for food, nutrisi in nutritional_values.items():
            if food not in [f[0] for f in self.meal_history] and food not in self.recommended_history:
                if (nutrisi["kalori"] <= remaining["remaining_calories"] and 
                    nutrisi["protein"] <= remaining["remaining_protein"] and 
                    nutrisi["lemak"] <= remaining["remaining_fat"] and 
                    nutrisi["karbohidrat"] <= remaining["remaining_carbs"]):
                    recommended_foods.append(food)
                else:
                    score = self.score_food(nutrisi, remaining)
                    scored_foods.append((food, score))

        if use_collaborative_filtering:
            collaborative_suggestion = self.collaborative_filtering(self.meal_history)
            print("Rekomendasi berdasarkan pola konsumsi serupa:", collaborative_suggestion)
            recommended_foods.extend(collaborative_suggestion)

        if recommended_foods:
            self.recommended_history.extend(recommended_foods)
            print("Makanan yang direkomendasikan:", recommended_foods)
        else:
            scored_foods.sort(key=lambda x: x[1], reverse=True)
            best_food_options = [food for food, score in scored_foods[:3]]
            print("Tidak ada makanan yang sepenuhnya memenuhi kriteria.")
            print("Makanan dengan kecocokan tertinggi yang direkomendasikan:", best_food_options)

        return recommended_foods

    def collaborative_filtering(self, user_meal_history):
        """
        Menyediakan rekomendasi berdasarkan kesamaan pola konsumsi dengan pengguna lain.

        Args:
            user_meal_history (list): Riwayat makanan pengguna

        Returns:
            list: Rekomendasi makanan dari pola pengguna lain yang serupa
        """
        similar_users = [
            ["Beijing Beef", "Fried Rice", "Super Greens"],
            ["Burger", "French Fries", "Sprite"],
            ["Chow Mein", "Honey Walnut Shrimp", "Water Spinach"]
        ]
        return random.choice(similar_users)

    def provide_feedback(self, food_name, accepted=True):
        """
        Menerima feedback dari pengguna mengenai makanan yang direkomendasikan.

        Args:
            food_name (str): Nama makanan yang diberikan sebagai rekomendasi
            accepted (bool): Apakah pengguna menerima rekomendasi atau tidak
        """
        if accepted:
            print(f"Pengguna menerima rekomendasi: {food_name}.")
        else:
            print(f"Pengguna menolak rekomendasi: {food_name}.")
            if food_name in self.recommended_history:
                self.recommended_history.remove(food_name)


class FoodRecommendationRNNWithFeedback(nn.Module):
    """
    Model RNN yang digunakan untuk mempelajari pola rekomendasi makanan berbasis data pengguna dan konsumsi.
    """
    
    def __init__(self, num_classes, input_size=15, hidden_size=128, num_layers=1):
        """
        Inisialisasi model RNN.

        Args:
            num_classes (int): Jumlah class makanan (output classes)
            input_size (int): Ukuran input
            hidden_size (int): Ukuran hidden layer
            num_layers (int): Jumlah layer pada RNN
        """
        super(FoodRecommendationRNNWithFeedback, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        """
        Forward pass dari input ke output prediksi makanan.

        Args:
            x (torch.Tensor): Input tensor

        Returns:
            torch.Tensor: Output dari model
        """
        h_0 = torch.zeros(1, x.size(0), 128)
        out, _ = self.rnn(x, h_0)
        out = self.fc(out[:, -1, :])
        return out


# --- Bagian Utama ---
if __name__ == "__main__":
    # Buat dataset dummy
    dummy_dataset = create_dummy_dataset_with_tdee()

    # Inisialisasi scaler untuk normalisasi data pengguna
    scaler = MinMaxScaler()
    user_data_numerical = np.array([[data["user_age"], data["user_weight"], data["user_height"], data["activity_level"]] for data in dummy_dataset])
    scaler.fit(user_data_numerical)

    # Contoh penggunaan dengan data pengguna dari dataset dummy
    user_data = dummy_dataset[0] 
    tdee = user_data["tdee"]

    complex_memory_feedback = ComplexDailyIntakeMemoryWithFeedback(
        target_calories=tdee,
        target_protein=150,  # Sesuaikan target makronutrien
        target_fat=70,
        target_carbs=250,
        target_fiber=30,
        target_vitamin_C=90,
        target_calcium=1000
    )

    # Contoh Input dan Rekomendasi
    print("\nInput sarapan: Chow Mein 200g pada jam 7:00")
    complex_memory_feedback.add_food("Chow Mein", 200, "sarapan")
    complex_memory_feedback.dynamic_adjustment("snack pagi", use_collaborative_filtering=True)

    print("\nInput makan siang: Burger 100g pada jam 12:30")
    complex_memory_feedback.add_food("Burger", 100, "makan siang")
    complex_memory_feedback.dynamic_adjustment("makan malam", use_collaborative_filtering=False)

    # Latihan Model RNN
    input_size = 15  # Menyertakan data pengguna
    hidden_size = 128
    num_layers = 1

    rnn_model_feedback = FoodRecommendationRNNWithFeedback(num_classes, input_size, hidden_size, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(rnn_model_feedback.parameters(), lr=0.001)

    num_epochs = 10
    for epoch in range(num_epochs):
        # Data training dummy.  Input harus dimodifikasi agar sesuai dengan data pengguna yang dinormalisasi.
        inputs = torch.randn(64, 5, input_size)  #  5 = sequence length (misalnya, 5 waktu makan dalam sehari)

        # Contoh integrasi data pengguna (dummy) -  Ganti dengan data asli dan normalisasi yang benar.
        user_data_batch = np.random.rand(64, 4)  # 4 fitur pengguna: age, weight, height, activity level
        user_data_normalized = scaler.transform(user_data_batch)
        user_data_tensor = torch.tensor(user_data_normalized, dtype=torch.float32).unsqueeze(1).repeat(1, 5, 1)  # Ulangi data pengguna untuk setiap timestep dalam sequence
        inputs[:, :, 11:] = user_data_tensor  # Tambahkan data pengguna ke input tensor

        labels = torch.randint(0, num_classes, (64,))
        outputs = rnn_model_feedback(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Contoh Prediksi
    user_data_prediction = dummy_dataset[10]
    user_data_numerical_prediction = np.array([[user_data_prediction["user_age"], user_data_prediction["user_weight"], user_data_prediction["user_height"], user_data_prediction["activity_level"]]])
    user_data_normalized_prediction = scaler.transform(user_data_numerical_prediction)
    user_data_tensor_prediction = torch.tensor(user_data_normalized_prediction, dtype=torch.float32).unsqueeze(1).repeat(1, 5, 1)

    user_sequence_input = torch.randn(1, 5, input_size)
    user_sequence_input[0, :, 11:] = user_data_tensor_prediction[0]

    output = rnn_model_feedback(user_sequence_input)
    predicted_food_class = torch.argmax(output, dim=1)
    print(f"\nRekomendasi makanan untuk pengguna dengan data {user_data_prediction}: {food_classes[predicted_food_class.item()]}")



Input sarapan: Chow Mein 200g pada jam 7:00

Kebutuhan sisa untuk snack pagi:
Kalori tersisa: 3001.99 kcal
Protein tersisa: 150.00 gram
Lemak tersisa: 70.00 gram
Karbohidrat tersisa: 250.00 gram
Serat tersisa: 30.00 gram
Vitamin C tersisa: 90.00 mg
Kalsium tersisa: 1000.00 mg
Rekomendasi berdasarkan pola konsumsi serupa: ['Chow Mein', 'Honey Walnut Shrimp', 'Water Spinach']
Makanan yang direkomendasikan: ['AW cola', 'Beijing Beef', 'Chow Mein', 'Honey Walnut Shrimp', 'Water Spinach']

Input makan siang: Burger 100g pada jam 12:30

Kebutuhan sisa untuk makan malam:
Kalori tersisa: 3001.99 kcal
Protein tersisa: 150.00 gram
Lemak tersisa: 70.00 gram
Karbohidrat tersisa: 250.00 gram
Serat tersisa: 30.00 gram
Vitamin C tersisa: 90.00 mg
Kalsium tersisa: 1000.00 mg
Tidak ada makanan yang sepenuhnya memenuhi kriteria.
Makanan dengan kecocokan tertinggi yang direkomendasikan: []

Rekomendasi makanan untuk pengguna dengan data {'user_age': 35, 'user_weight': 99, 'user_height': 160, 'user_gende